## First, imports:

In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy import units

Import my library:

In [ ]:
import os
import sys

apt_path = os.path.abspath(os.path.join("..", "apostletools"))
sys.path.append(apt_path)

import snapshot
import dataset_comp

In [ ]:
import importlib
importlib.reload(snapshot)
importlib.reload(dataset_comp)

# Mass Distribution of Subhalos

To visualize the mass distribution, at $z=0$, I plot a simple subhalo count accumulation curve, which at any point on the mass axis gives the total number of subhalos (or of satellite or isolated galaxies) with masses larger than the given mass. Mass is measured by $v_\mathrm{max} = \max_{r} \sqrt{\frac{G M(<r)}{r}}$.

## Motivation

Plenty of interesting and instructive observations can be made from the mass distribution figure alone. For the curv-p08* simulation, we expect find an indication of inhibited structure formation towards the small scales, relative to the plain-lcdm. However, on the larger scales, the curves from different simulations should approach each other (although random effects from low number counts also come into play). 

But the absence of small-scale power in the initial physical power spectrum means that the component of power that is due to numerical noise also becomes more significant --- and, indeed, will dominate on small enough scales (shot noise becomes more relevant towards the resolution limits of the simulation). This is expected to be visible in the mass function as well.

Furthermore, the mass functions satellite galaxies and isolated galaxies in any simulations should be expected to differ. 

From the single plot alone, it is impossible to make the connection between any particular feature of the mass function and any of the above-mentioned potential causes. Of course, we are also only looking at a single simulated instance of the LG.

---

## Set Parameters for the Plots

Choose the snapshot and the simulations, and define M31 and MW in each simulation:

Create a dictionary of the datasets from each simulation. 

In [ ]:
snap_id = 127
data = {
    "plain-LCDM-LR": {
        "Snapshot": snapshot.Snapshot("V1_LR_fix", snap_id),
        "M31_ID": (1, 0),
        "MW_ID": (2, 0),
        "Color": ["gray"]
    },
    "curv-p082-LR": {
        "Snapshot": snapshot.Snapshot("V1_LR_curvaton_p082_fix", snap_id),
        "M31_ID": (1, 0),
        "MW_ID": (1, 1),
        "Color": ["pink"]
    },
    "curv-p084-LR": {
        "Snapshot": snapshot.Snapshot("V1_LR_curvaton_p084_fix", snap_id),
        "M31_ID": (1, 0),
        "MW_ID": (1, 0),
        "Color": ["lightblue"]
    },
    "plain-LCDM": {
        "Snapshot": snapshot.Snapshot("V1_MR_fix", snap_id),
        "M31_ID": (1, 0),
        "MW_ID": (2, 0),
        "Color": ["black", "gray"]
    },
    "curv-p082": {
        "Snapshot": snapshot.Snapshot("V1_MR_curvaton_p082_fix", snap_id),
        "M31_ID": (1, 0),
        "MW_ID": (1, 1),
        "Color": ["red", "pink"]
    }
}

---

## Retrieve Data

### Create a Dictionary

For easy handling of the relevant data, define a data dictionary that, at the top level, has entries for all simulations. Under each simulation sim_data, add items for the needed datasets and, under the "Selections" key, a sub-dictionary of masking arrays for each needed condition (e.g. satellite, luminous, $v_\mathrm{max}$ inside range, etc.).

First, add the above definitions into the data dict:

Then, loop over simulations, retrieve data, compute masking arrays, and add to the dictionary:

In [ ]:
for key, sim_data in data.items():    
    # Get data:
    snap = sim_data["Snapshot"]
    v1kpc = snap.get_subhalos("V1kpc", "Extended") * units.cm.to(units.km)
    
    # Compute masking arrays:
    mask_m31, mask_mw, mask_isol = dataset_comp.split_satellites_by_distance(
        snap, sim_data["M31_ID"], sim_data["MW_ID"]
    )
    mask_sat = np.logical_or(mask_m31, mask_mw)
    mask_lum, mask_dark = dataset_comp.split_luminous(snap)
    mask_vmax = dataset_comp.prune_vmax(snap, low_lim=10)

    # Sort by vmax and add a dummy point with very small vmax 
    # (to continue the curves to the y-axis):
    sim_data["V1kpc"] = v1kpc
    
    data[key]["Selections"] = {
        "M31": mask_m31,
        "MW": mask_mw,
        "Satellite": mask_sat,
        "Isolated": mask_isol,
        "Luminous": mask_lum,
        "Dark": mask_dark,
        "Vmax": mask_vmax
    }

## Plot Only Total Counts

In [ ]:
# Set some parameters:
x_down = 5; x_up = 80
y_down = 1; y_up = 800

# Choose font sizes:
parameters = {'axes.titlesize': 10,
              'axes.labelsize': 8,
              'xtick.labelsize': 8,
              'ytick.labelsize': 8,
              'legend.fontsize': 8,
              'legend.title_fontsize': 7}

# Set fonts:
plt.rcParams.update(parameters)
plt.tight_layout()

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(6, 3), sharey="row")
plt.subplots_adjust(wspace=0)

# Set axis:
axes[0].set_ylim(0, 1.2)
axes[0].set_ylabel("$N(>v_{\mathrm{1\,\mathrm{kpc}}})$")
for ax in axes:
    ax.set_xlim(x_down, x_up)
    ax.set_xlabel("$v_{\mathrm{1\,\mathrm{kpc}}}[\mathrm{km s^{-1}}]$")
    ax.set_xscale("log")
    # ax.set_yscale("log")
    
axes[0].set_title("Satellite galaxies")
axes[1].set_title("Isolated galaxies")

for (sim_name, sim_data) in data.items():
               
    v1kpc = sim_data["V1kpc"]
        
    # ISOLATED GALAXIES
    # -----------------
    
    mask_isol = np.logical_and(sim_data["Selections"]["Isolated"],
                               sim_data["Selections"]["Vmax"])
    n_isol = np.sum(mask_isol) - 1    
    x = v1kpc[mask_isol]
    y = np.append(1, np.arange(1, n_isol + 1)[::-1] / n_isol)
    
    x = np.append(x, x[-1])
    y = np.append(y, 0)
    axes[1].plot(x, y, c=sim_data["Color"][0], 
                 linestyle="solid", label=sim_name)


    # SATELLITES
    # ----------
        
    if sim_name.split("-")[1] == "p084":
        continue

    mask_sat = np.logical_and(sim_data["Selections"]["Satellite"],
                              sim_data["Selections"]["Vmax"])
    n_sat = np.sum(mask_sat) - 1    
    x = v1kpc[mask_sat]
    y = np.append(1, np.arange(1, n_sat + 1)[::-1] / n_sat)
    
    x = np.append(x, x[-1])
    y = np.append(y, 0)
    axes[0].plot(x, y, c=sim_data["Color"][0], linestyle="solid",
                 label=sim_name)
        
axes[1].legend()
    
plt.tight_layout()

### Save the Figure

In [ ]:
filename = "v1kpc_distribution_total.png"
    
path = os.path.abspath(os.path.join("..", "Figures", "MediumResolution"))
filename = os.path.join(path, filename)

fig.savefig(filename, dpi=300, bbox_inches="tight")

## Plot Luminous and Dark

In [ ]:
def count_curve(arr):
    
    arr.sort()
    
    n = arr.size
    x = np.append(10**-5, np.append(arr, arr[-1]))
    y = np.append(1, np.append(np.arange(1, n + 1)[::-1] / n, 0))
    
    return x, y

In [ ]:
print(count_curve(np.arange(1,15)))

In [ ]:
# Set some parameters:
x_down = 5; x_up = 80
y_down = 1; y_up = 800

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(6, 3), sharey="row")
plt.subplots_adjust(wspace=0)

# Set axis:
axes[0].set_ylim(0, 1.2)
axes[0].set_ylabel("$f\,(>v_{\mathrm{1\,\mathrm{kpc}}})$")
for ax in axes:
    ax.set_xlim(2, 60)
    ax.set_xlabel("$v_{\mathrm{1\,\mathrm{kpc}}}[\mathrm{km s^{-1}}]$")
    # ax.set_xscale("log")
    # ax.set_yscale("log")
    
axes[0].set_title("Satellite galaxies")
axes[1].set_title("Isolated galaxies")

# Plot only MR:
data_mr = {key: data[key] for key in data.keys() 
           & {'plain-LCDM', 'curv-p082'}}
for (sim_name, sim_data) in data_mr.items():
               
    v1kpc = sim_data["V1kpc"]
            
    # SATELLITES
    # ----------
    
    mask_sat = np.logical_and(sim_data["Selections"]["Satellite"],
                              sim_data["Selections"]["Vmax"])
    
    # Plot dark luminous:
    mask_dark = np.logical_and(mask_sat, sim_data["Selections"]["Dark"])
    x, y = count_curve(v1kpc[mask_dark])
    axes[0].plot(x, y, c=sim_data["Color"][0], linestyle="solid")
    
#     # Plot only luminous satellites:
    mask_lum = np.logical_and(mask_sat, sim_data["Selections"]["Luminous"])
    x, y = count_curve(v1kpc[mask_lum])
    axes[0].plot(x, y, c=sim_data["Color"][0], linestyle="dashed")

        
#     # ISOLATED GALAXIES
#     # -----------------
    
    mask_isol = np.logical_and(sim_data["Selections"]["Isolated"],
                              sim_data["Selections"]["Vmax"])
    
    # Plot dark isolated:
    mask_dark = np.logical_and(mask_isol, sim_data["Selections"]["Dark"])
    x, y = count_curve(v1kpc[mask_dark])
    axes[1].plot(x, y, c=sim_data["Color"][0], linestyle="solid")
    
    # Plot only luminous satellites:
    mask_lum = np.logical_and(mask_isol, sim_data["Selections"]["Luminous"])
    x, y = count_curve(v1kpc[mask_lum])
    axes[1].plot(x, y, c=sim_data["Color"][0], linestyle="dashed")
        
# # Make dummy plots for the legend:   
# l = []
# for entry in data_mr.values():
#     l_dum, = axes[1].plot([], [], c=entry['Color'][0], linestyle="solid")
#     l.append(l_dum)
# sim_legend = axes[1].legend(l, list(data_mr.keys()), loc="upper right")
# axes[1].add_artist(sim_legend)

# axes[0].plot([], [], c='k', linestyle="solid", label="All")
# axes[0].plot([], [], c='k', linestyle="dashed", label="Luminous")
# axes[0].legend(loc="upper right")

plt.tight_layout()

### Save the Figure

In [ ]:
filename = "v1kpc_distribution_norm.png"
    
path = os.path.abspath(os.path.join("..", "Figures", "MediumResolution"))
filename = os.path.join(path, filename)

fig.savefig(filename, dpi=300, bbox_inches="tight")